In [1]:
import os
from numpy._distributor_init import NUMPY_MKL 
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from threading import Thread
from imutils import face_utils
import numpy as np
import playsound
import imutils
import dlib
import cv2

In [ ]:
def eye_aspect_ratio(eye):
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        C = dist.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear

In [ ]:
det = dlib.get_frontal_face_detector()
pre = dlib.shape_predictor('shape_predictor.dat')
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(cStart, cEnd) = (6, 11)
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]
(fStart, fEnd) = (1, 27)

In [ ]:
train_dir = 'data'
pos_v = []
neg_v = []

for v_name in os.listdir(train_dir):
    v_path = os.path.join(train_dir, v_name)
    v = cv2.VideoCapture(v_path)
    fps = v.get(cv2.CAP_PROP_FPS)
    length = v.get(cv2.CAP_PROP_FRAME_COUNT)
    if 'pos' in v_name:
            pos_v.append(v)
    elif 'neg' in v_name:
            neg_v.append(v)
        
print("Positive images #: ", len(pos_v))
print("Negative images #: ", len(neg_v))
rezultat=[]

In [ ]:
def im_det(videoSn):
    ret=True;
    countN=0
    ukZ=0
    countZ=0
    COUNTER=0
    TOTAL=0
    rezultat=[]
    v=cv2.VideoCapture(0)
    fps = videoSn.get(cv2.CAP_PROP_FPS)
    length = videoSn.get(cv2.CAP_PROP_FRAME_COUNT)
    while ret:
    
        ret, frame=videoSn.read()

        if(ret):


            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            rects = detector(gray, 0)

            for rect in rects:
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)

           
                leftEye = shape[lStart:lEnd]
                rightEye = shape[rStart:rEnd]
                chin= shape[cStart:cEnd]
                mouthU=shape[50:53]
                mouthD=shape[56:59]
                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)


            
                ear = (leftEAR + rightEAR) / 2.0
                leftEyeHull = cv2.convexHull(leftEye)
                rightEyeHull = cv2.convexHull(rightEye)
                chinH=cv2.convexHull(chin)
                mouthH=cv2.convexHull(mouthU)
                mouthH2=cv2.convexHull(mouthD)
                
                cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [chinH], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [mouthH], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [mouthH2], -1, (0, 255, 0), 1)
                
                distance=abs(mouthU[2]-mouthD[2])

                distanceE=abs(leftEye[3]-leftEye[0])


                if ((abs(distance[0]-distance[1]))> 35 and distanceE[0]>25) or ((abs(distance[0]-distance[1]))> 11 and distanceE[0]>11) or ((abs(distance[0]-distance[1]))>4 and distanceE[0]<12):
                    countZ+=1
                    countN=0
                else:     
                    if (countZ>20 and countN>0):
                        ukZ+=1
                        countZ=0
                        countN=0
                    if(countN>3):
                        countZ=0
                    countN+=1
                if (ear < 0.20 and distanceE[0]<30) or (ear < 0.23 and distanceE[0]>30):
                    COUNTER += 1
                    cv2.putText(frame, "Eye closed", (170, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


                else:
                    if COUNTER >=2:
                        TOTAL += 1
                        COUNTER = 0
                        
                    COUNTER = 0
                    cv2.putText(frame, "Eye open", (170, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            
                cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "Z: {}".format(ukZ), (100, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        #cv2.imshow("Frame", frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    v.release()
    cv2.destroyAllWindows()
    rezultat.append([TOTAL, ukZ])
    print("Treptaji ",TOTAL)
    print("Zevanje ",ukZ)
    return([TOTAL, ukZ])

In [ ]:
pos_r = []
neg_r = []
labels=[]
pom=[]

for v in pos_v:
    pom=im_det(v)
    print("treptaji ", pom[0])
    print("zevanje ", pom[1])
    pos_r.append(pom)
    labels.append(1)
    
    
for v in neg_v:
    neg_r.append(im_det(v))
    labels.append(0)

pos_r = np.array(pos_r)
print("Positive images #: ", len(pos_r))
neg_r = np.array(neg_r)
print("neg_features images #: ", len(neg_r))
x = np.vstack((pos_r, neg_r))
y = np.array(labels)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=3)
print('Train shape: ', x_train.shape, y_train.shape)
print('Test shape: ', x_test.shape, y_test.shape)

In [ ]:
clf_svm = SVC(kernel='linear', probability=True) 
clf_svm.fit(x_train, y_train)
y_train_pred = clf_svm.predict(x_train)
y_test_pred = clf_svm.predict(x_test)
print("Train accuracy: ", accuracy_score(y_train, y_train_pred))
print("Validation accuracy: ", accuracy_score(y_test, y_test_pred))

In [ ]:
#Nastavak je za praćenje frejmova preko kamere i pojedinačnih video snimaka

In [ ]:
vs = cv2.VideoCapture('data/10-FemaleNoGlasses-Yawning-pos.avi')
print("Ucitava se ", vs)
fps = vs.get(cv2.CAP_PROP_FPS)
print("Broj frejmova po sekundi u videu ", fps)
ret=True;
countN=0
ukZ=0
countZ=0
COUNTER=0
TOTAL=0

In [ ]:
while ret:
    
    ret, frame=vs.read()
    
    if(ret):
        (h, w) = frame.shape[:2]

        center = (1, 1)
        angle = 0
        scale =0.5
        M = cv2.getRotationMatrix2D(center, angle, scale)
        frame =cv2.warpAffine(frame, M, (h, w))
        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)
        
        for rect in rects:
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)

           
                leftEye = shape[lStart:lEnd]
                rightEye = shape[rStart:rEnd]
                chin= shape[cStart:cEnd]
                mouthU=shape[50:53]
                mouthD=shape[56:59]
                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)


            
                ear = (leftEAR + rightEAR) / 2.0
                leftEyeHull = cv2.convexHull(leftEye)
                rightEyeHull = cv2.convexHull(rightEye)
                chinH=cv2.convexHull(chin)
                mouthH=cv2.convexHull(mouthU)
                mouthH2=cv2.convexHull(mouthD)
                
                cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [chinH], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [mouthH], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [mouthH2], -1, (0, 255, 0), 1)
                
                distance=abs(mouthU[2]-mouthD[2])

                distanceE=abs(leftEye[3]-leftEye[0])


                if ((abs(distance[0]-distance[1]))> 35 and distanceE[0]>25) or ((abs(distance[0]-distance[1]))> 11 and distanceE[0]>11) or ((abs(distance[0]-distance[1]))>4 and distanceE[0]<12):
                    
                    countZ+=1
                    countN=0
                else:
                    
                    if (countZ>20 and countN>0):
                        ukZ+=1
                        countZ=0
                        countN=0
                    if(countN>3):
                        countZ=0
                    countN+=1
                if (ear < 0.20 and distanceE[0]<30) or (ear < 0.23 and distanceE[0]>30):
                    COUNTER += 1
                    cv2.putText(frame, "Eye closed", (170, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    if(COUNTER/fps>2):
                        playsound.playsound('alarm.wav')


                else:
                    if COUNTER >=2:
                        print("Eye been closed for %d", COUNTER)
                        TOTAL += 1
                        COUNTER = 0
                        
                    COUNTER = 0
                    cv2.putText(frame, "Eye open", (170, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            
                cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "Z: {}".format(ukZ), (100, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        cv2.imshow("Frame", frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
#vs.stop()
vs.release()
cv2.destroyAllWindows()
print("Blinks: {}",TOTAL )
print("y: {}",ukZ )

In [ ]:
#Za kameru 
vs = VideoStream().start()
ret=True;
countN=0
ukZ=0
countZ=0
COUNTER=0
TOTAL=0

In [ ]:
while True:
    
    frame=vs.read()
    
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
        
    for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

           
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            chin= shape[cStart:cEnd]
            mouthU=shape[50:53]
            mouthD=shape[56:59]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)


            
            ear = (leftEAR + rightEAR) / 2.0
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            chinH=cv2.convexHull(chin)
            mouthH=cv2.convexHull(mouthU)
            mouthH2=cv2.convexHull(mouthD)
                
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [chinH], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [mouthH], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [mouthH2], -1, (0, 255, 0), 1)
                
            distance=abs(mouthU[2]-mouthD[2])

            distanceE=abs(leftEye[3]-leftEye[0])
            
            distanceC=abs(chinH[1]-0)
            if ((abs(distance[0]-distance[1]))> 35 and distanceE[0]>25) or ((abs(distance[0]-distance[1]))> 11 and distanceE[0]>11) or ((abs(distance[0]-distance[1]))>6 and distanceE[0]<12):
                    
                countZ+=1
                countN=0
            else:
                countN+=1
                if (countZ>20 and countN>3):
                    ukZ+=1
                    countZ=0
                    countN=0
                if(countN>3):
                    countZ=0
                    
            if (ear < 0.20 and distanceE[0]<30) or (ear < 0.23 and distanceE[0]>30):
                COUNTER += 1
                cv2.putText(frame, "Eye closed", (170, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                if(COUNTER/fps>2):
                    playsound.playsound('alarm.wav')
                elif distanceC[0][1]<250:
                    playsound.playsound('alarm.wav')

            else:
                if COUNTER >=2:
                    print("Eye been closed for %d", COUNTER)
                    TOTAL += 1
                    COUNTER = 0
                        
                COUNTER = 0
                cv2.putText(frame, "Eye open", (170, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            
            cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "Z: {}".format(ukZ), (100, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
vs.stop()
cv2.destroyAllWindows()
print("Blinks: {}",TOTAL )
print("y: {}",ukZ )